In [1]:
!pip install scrapy
!pip install response

     |████████████████████████████████| 245kB 2.8MB/s 
     |████████████████████████████████| 235kB 8.8MB/s 
     |████████████████████████████████| 3.2MB 8.2MB/s 
     |████████████████████████████████| 2.7MB 29.5MB/s 
     |████████████████████████████████| 3.1MB 44.4MB/s 
     |████████████████████████████████| 61kB 6.8MB/s 
     |████████████████████████████████| 61kB 7.2MB/s 
  Created wheel for protego: filename=Protego-0.1.16-cp36-none-any.whl size=7765 sha256=f26fe338965a495d32a9c5b9abd0ed87a5c053597c5ee629f39d6942c089d419
  Stored in directory: /root/.cache/pip/wheels/51/01/d1/4a2286a976dccd025ba679acacfe37320540df0f2283ecab12
  Created wheel for PyDispatcher: filename=PyDispatcher-2.0.5-cp36-none-any.whl size=11515 sha256=e3cd5eb14a8e39a0ecca49506c1921f02bc56eb159cd260a0733d7966b738571
  Stored in directory: /root/.cache/pip/wheels/88/99/96/cfef6665f9cb1522ee6757ae5955feedf2fe25f1737f91fa7f
Successfully built protego PyDispatcher


In [2]:
import time
import response
import requests
import pandas as pd
from scrapy.http import TextResponse

In [10]:
# Problem 1 – scraping quotes

In [3]:
URL = "http://quotes.toscrape.com/"
base_url = "http://quotes.toscrape.com"

In [4]:
class Quotes:

    def __init__(self,URL):
        self.URL = URL
        self.page = requests.get(self.URL)
        self.response = TextResponse(body=self.page.text,url=self.URL,encoding="utf-8")

    def get_quotes(self):
        quotes = self.response.css("span.text::text").extract()
        authors = self.response.css("small.author::text").extract() 
        tags = [i.css("a.tag::text").extract() for i in self.response.css("div.tags")]
        hyperlinks = [base_url+i for i in self.response.css("small.author ~ a::attr(href)").extract()]
        return quotes, authors, tags, hyperlinks

    def get_next(self):
        next_url = self.response.css("li.next a::attr(href)").extract()
        return next_url

In [7]:
quotes = []
Qts = Quotes(URL)

In [8]:
while True:
    time.sleep(2)
    if(Qts.get_next()==[]):
        quotes.append(Qts.get_quotes())
        break
    else:
        quotes.append(Qts.get_quotes())
        URL = base_url + Qts.get_next()[0]
        Qts = Quotes(URL)

quotes

[(['“The truth." Dumbledore sighed. "It is a beautiful and terrible thing, and should therefore be treated with great caution.”',
   "“I'm the one that's got to die when it's time for me to die, so let me live my life the way I want to.”",
   '“To die will be an awfully big adventure.”',
   '“It takes courage to grow up and become who you really are.”',
   '“But better to get hurt by the truth than comforted with a lie.”',
   '“You never really understand a person until you consider things from his point of view... Until you climb inside of his skin and walk around in it.”',
   '“You have to write the book that wants to be written. And if the book will be too difficult for grown-ups, then you write it for children.”',
   '“Never tell the truth to people who are not worthy of it.”',
   "“A person's a person, no matter how small.”",
   '“... a mind needs books as a sword needs a whetstone, if it is to keep its edge.”'],
  ['J.K. Rowling',
   'Jimi Hendrix',
   'J.M. Barrie',
   'E.E. Cum

In [11]:
# Problem 2 – scraping movies

In [12]:
# Scrapes  the top 100 movies from the following page: https://www.imdb.com/chart/moviemeter/.

In [13]:
URL = "https://www.imdb.com/chart/moviemeter/"
base_url = "https://www.imdb.com"

In [14]:
class Movies: 

    def __init__(self,URL):
        self.URL = URL
        self.page = requests.get(self.URL)
        self.response = TextResponse(body=self.page.text,url=self.URL,encoding="utf-8")

    def get_movies(self):
        titles = self.response.css("td.titleColumn>a::text").extract()
        year = self.response.css("td.titleColumn>span.secondaryInfo::text").extract()
        ranking = [i.css("div.velocity::text").extract() for i in self.response.css("td.titleColumn")]
        rating_td = self.response.css('td[class = "ratingColumn imdbRating"]')
        rating = [i.css('strong::text').extract() for i in rating_td]
        rel_hyperlinks = self.response.css("td.titleColumn > a::attr(href)").extract()
        hyperlink_movie = [base_url + i for i in rel_hyperlinks]
        return pd.DataFrame({"Titles":titles,"Year":year,"Hyperlink_movie":hyperlink_movie,"Ranking":ranking,"Rating":rating})

In [15]:
movies = Movies("https://www.imdb.com/chart/moviemeter/")

movies.get_movies()

,Titles,Year,Hyperlink_movie,Ranking,Rating
0,365 Days,(2020),https://www.imdb.com/title/tt10886166/,[1\n(no change)\n],[3.5]
1,Da 5 Bloods,(2020),https://www.imdb.com/title/tt9777644/,"[2\n, \n]",[6.7]
2,Artemis Fowl,(2020),https://www.imdb.com/title/tt3089630/,"[3\n, \n]",[4.1]
3,The King of Staten Island,(2020),https://www.imdb.com/title/tt9686708/,"[4\n, \n]",[7.2]
4,Knives Out,(2019),https://www.imdb.com/title/tt8946378/,"[5\n, \n]",[7.9]
...,...,...,...,...,...
95,The Rising Hawk,(2019),https://www.imdb.com/title/tt7439064/,"[96\n, \n]",[6.2]
96,Bloodshot,(2020),https://www.imdb.com/title/tt1634106/,"[97\n, \n]",[5.7]
97,The Room,(2003),https://www.imdb.com/title/tt0368226/,"[98\n, \n]",[3.7]
98,Emma.,(2020),https://www.imdb.com/title/tt9214832/,"[99\n, \n]",[6.8]


In [16]:
# Problem 3 – scraping books 

In [19]:
URL = "http://books.toscrape.com/"
base_url = "http://books.toscrape.com"

In [21]:
class Books:

    def __init__(self,URL):
        self.URL = URL
        self.page = requests.get(self.URL)
        self.response = TextResponse(body=self.page.text,url=self.URL,encoding="utf-8")
    def get_books(self):  
        page = requests.get(URL)
        response = TextResponse(body=page.text,url=URL,encoding="utf-8")
        title = self.response.css("h3 a::attr(title)").extract()
        price = self.response.css("p.price_color::text").extract()
        Price = [i.replace("Â","") for i in price]
        stock = [i.css("p.instock.availability::text").extract() for i in self.response.css("div.product_price")]
        rel_hyperlinks = self.response.css("h3 a::attr(href)").extract()
        hyperlink_book = [base_url + i for i in rel_hyperlinks]
        rel_hyperlink = self.response.css("img.thumbnail::attr(src)").extract()
        hyperlink_image = [base_url + i for i in rel_hyperlink]
        rating_books = self.response.css("p.star-rating::attr(class)").extract()
        Rating_books = [i.replace("star-rating","") for i in rating_books]
        return {"Title":title,"Price":Price,"Stock":stock,"Hyperlink_book":hyperlink_book,"Hyperlink_image":hyperlink_image,"Rating":Rating_books}

In [23]:
books = []
Bks = Books(URL)

In [24]:
for i in range(1,51):
  books.append(Bks.get_books())
  URL = base_url
  Bks = Books(URL)

books


[{'Hyperlink_book': ['http://books.toscrape.comcatalogue/a-light-in-the-attic_1000/index.html',
   'http://books.toscrape.comcatalogue/tipping-the-velvet_999/index.html',
   'http://books.toscrape.comcatalogue/soumission_998/index.html',
   'http://books.toscrape.comcatalogue/sharp-objects_997/index.html',
   'http://books.toscrape.comcatalogue/sapiens-a-brief-history-of-humankind_996/index.html',
   'http://books.toscrape.comcatalogue/the-requiem-red_995/index.html',
   'http://books.toscrape.comcatalogue/the-dirty-little-secrets-of-getting-your-dream-job_994/index.html',
   'http://books.toscrape.comcatalogue/the-coming-woman-a-novel-based-on-the-life-of-the-infamous-feminist-victoria-woodhull_993/index.html',
   'http://books.toscrape.comcatalogue/the-boys-in-the-boat-nine-americans-and-their-epic-quest-for-gold-at-the-1936-berlin-olympics_992/index.html',
   'http://books.toscrape.comcatalogue/the-black-maria_991/index.html',
   'http://books.toscrape.comcatalogue/starving-hearts-t